**Import dependencies**

In [ ]:
import os
import fastprogress
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision

from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau, ExponentialLR, StepLR


In [ ]:
use_cuda = True

In [ ]:
if use_cuda and not torch.cuda.is_available():
    print("Error: cuda requested but not available, will use cpu instead!")
    device = torch.device('cpu')
elif not use_cuda:
    print("Info: will use cpu!")
    device = torch.device('cpu')
else:
    print("Info: cuda requested and available, will use gpu!")
    device = torch.device('cuda:0')


**Import Dataset**



In [ ]:
## UPLOAD CSV-FILE HERE ##
from google.colab import files
uploaded = files.upload() # import daily_data.csv

In [ ]:
uploaded = files.upload() # import holidays_for_daily.csv

In [ ]:
data = pd.read_csv('daily_data.csv',sep = ",")
holidays = pd.read_csv('holidays_for_daily.csv',sep = ",")

**Data preparation**

In [ ]:
ts = data['TS'].values.astype(float)
og = data['OG'].values.astype(float)
fd = data['FD'].values.astype(float)
ff = data['FF'].values.astype(float)

In [ ]:
product = ff
product = np.asarray(product)

max_value = 65000
timeseries_normalized = product / max_value
timeseries_normalized = torch.FloatTensor(timeseries_normalized).view(-1)

hol_arr = np.asarray(holidays)
hol_tens = torch.FloatTensor(hol_arr[:,1:])

In [ ]:
zeros = torch.zeros((1,14))
hol_shift = torch.cat((hol_tens,zeros),0)
hol_shift = hol_shift[1:]
total = torch.cat((timeseries_normalized.reshape((len(hol_tens),1)),hol_shift),dim = 1)

**Data Preprocessing**



Create X (training sequence) and y (training label) in order to feed LSTM net

In [ ]:
def create_inout_sequences(input_data, tw, pred_length):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw-pred_length+1):
        train_seq = input_data[i:i+tw+pred_length-1]
        train_label = input_data[i+tw : i+tw+pred_length,0]

        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [ ]:
'Create inout_seq'
train_window = 365
pred_length = 42
inout_seq = create_inout_sequences(total, train_window, pred_length)
display(len(inout_seq))


Train, validation and test split

In [ ]:
test_set = inout_seq[-(333):]
val_set = inout_seq[-770:-(363+pred_length)]
train_set = inout_seq[:-(770+pred_length)]


display(len(train_set))
display(len(val_set))
display(len(test_set))

**Creating LSTM model**

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, drop_out,num_layers = 2, pred_length = 16, device = device):
        super(LSTM, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.drop_out = drop_out
        self.device = device
        self.pred_length = pred_length
        self.decoder = DecoderCell(input_dim, hidden_dim, num_layers, drop_out)

        self.lstm = nn.LSTM(
            input_size = input_dim,
            hidden_size = hidden_dim,
            num_layers = num_layers,
            dropout = self.drop_out if self.num_layers > 1 else 0,
            batch_first = True)

    
    def reset_hidden_state(self,input):
        self.hidden = (torch.zeros(self.num_layers, input, self.hidden_dim).to(self.device),
                       torch.zeros(self.num_layers, input, self.hidden_dim).to(self.device))
    
    def detach_hidden_state(self):
        self.hidden = ( self.hidden[0].detach().to(self.device), self.hidden[1].detach().to(self.device) )

    def forward(self, input):

        lstm_out, self.hidden = self.lstm(input[:,:-(self.pred_length - 1),:], (self.hidden[0][:,:input.size()[0],:].contiguous(),self.hidden[1][:,:input.size()[0],:].contiguous()))
        output, self.hidden = self.decoder(input[:,-self.pred_length,:].reshape((input.size()[0],1, self.input_dim)), self.hidden)
        output = output.reshape((input.size()[0],1))

        output_iter = output


        for i in range(1, self.pred_length):
          new_input = torch.cat((output_iter, input[:,-(self.pred_length-i),1:]), dim = 1)
          output_iter, self.hidden = self.decoder(new_input.reshape((input.size()[0],1, self.input_dim)), self.hidden)
          output_iter = output_iter.reshape((input.size()[0],1))
          output = torch.cat((output, output_iter), dim = 1)
        return output




In [ ]:
class DecoderCell(nn.Module):
    def __init__(self, input_feature_len, hidden_size, n_layers, dropout=0.2):
        super().__init__()
        self.n_layers = n_layers
        self.drop_out = dropout
        self.lstm = nn.LSTM(
            input_size=input_feature_len,
            hidden_size=hidden_size,
            num_layers=self.n_layers,
            dropout = self.drop_out if self.n_layers > 1 else 0,
            batch_first = True
        )
        self.out = nn.Linear(hidden_size, 1)
        

    def forward(self, y, prev_hidden):
        lstm_out, hidden = self.lstm(y, prev_hidden)
        output = self.out(lstm_out)
        return output, hidden

**Function to train the model**

In [ ]:
def train (dataloader, optimizer,model,loss_fn, master_bar, device = device ):

    epoch_loss = []

    for seq, labels in fastprogress.progress_bar(dataloader, parent=master_bar):
        model.reset_hidden_state(seq.size()[0])
        seq, labels = seq.to(device),labels.to(device)
        optimizer.zero_grad()

        model.train()
        
        # Forward
        y_pred = model(seq)

        # Compute loss
        single_loss = loss_fn(y_pred.to(device), labels)
        single_loss.backward(retain_graph = True)
        # Training step
        optimizer.step()

        epoch_loss.append(single_loss.item())
    return np.mean(epoch_loss)

**Function to validate the model**

In [ ]:
def validate(dataloader, model, loss_fn,master_bar, device = device):
    """Compute loss on validation set."""

  
    epoch_loss = []
    predictions = []   

    model.eval()
    with torch.no_grad():
        for seq, labels in fastprogress.progress_bar(dataloader, parent=master_bar):
            model.reset_hidden_state(seq.size()[0])
            seq, labels = seq.to(device),labels.to(device)
            # make a prediction on validation set
            y_pred = model(seq)
            predictions.append(y_pred)
            # Compute loss
            single_loss = loss_fn(y_pred, labels)
            epoch_loss.append(single_loss.item())

            

    return np.mean(epoch_loss),predictions

**Function to run training**

In [ ]:
def run_training(model, optimizer, loss_fn, num_epochs, 
                train_dataloader,val_dataloader,verbose=True, early_stopper = True, lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau):
    """ Run model training """
    
    start_time = time.time()
    master_bar = fastprogress.master_bar(range(num_epochs))
    train_losses, val_losses = [],[]
    scheduler = lr_scheduler(optimizer, 'min', patience = 5, factor=0.5)

     

    for epoch in master_bar:


        # train model
        epoch_train_loss = train(train_dataloader, optimizer, model, loss_fn,master_bar)
        # validate model
        epoch_val_loss,y_pred = validate(val_dataloader, model, loss_fn,master_bar)

        # Save loss for plotting
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)

        if verbose:
            master_bar.write(f'Train loss: {epoch_train_loss:.3f}, val loss: {epoch_val_loss:.3f}')
        
        scheduler.step(epoch_val_loss)

        if early_stopper:
           early_stopper.update(epoch_val_loss, model)
           if early_stopper.early_stop:
             model = early_stopper.load_checkpoint(model)
             break


    time_elapsed = np.round(time.time() - start_time, 0).astype(int)
    print(f'Finished training after {time_elapsed} seconds.')
    return model, train_losses, val_losses, y_pred


**Function to plot learning curves**

In [ ]:
def plot(title, label, train_results, val_results, yscale='linear', extra_pt=None, extra_pt_label = None):
    
    """Plot learning curves"""
    
    epoch_array = np.arange(len(train_results)) + 1
    train_label, val_label = "Training "+label.lower(), "Validation "+label.lower()
    
    sns.set(style='ticks')

    plt.plot(epoch_array, train_results, epoch_array, val_results, linestyle='dashed', marker='o')
    legend = ['Train results', 'Validation results']
        
    if extra_pt:
        plt.plot(extra_pt[0],extra_pt[1],marker = '*', color = 'k')
        plt.annotate(extra_pt_label,extra_pt)

    plt.legend(legend)
    plt.xlabel('Epoch')
    plt.ylabel(label)
    plt.yscale(yscale)
    plt.title(title)
    
    sns.despine(trim=True, offset=5)
    plt.title(title, fontsize=15)

    plt.show()

**Early Stopping**

In [ ]:
class EarlyStopper:
    """Early stops the training if validation accuracy does not increase after a
    given patience.
    """
    def __init__(self, verbose=False, path='checkpoint.pt', patience=10):
        """Initialization.

        Args:
            verbose (bool, optional): Print additional information. Defaults to False.
            path (str, optional): Path where checkpoints should be saved. 
                Defaults to 'checkpoint.pt'.
            patience (int, optional): Number of epochs to wait for increasing
                accuracy. If accyracy does not increase, stop training early. 
                Defaults to 1.
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.__early_stop = False
        self.val_loss_min = np.Inf
        self.path = path
        
        
    @property
    def early_stop(self):
        """True if early stopping criterion is reached.

        Returns:
            [bool]: True if early stopping criterion is reached.
        """

        if self.patience == self.counter:
          return True
        else:
          return(False)

        
        
    def update(self, val_loss, model):
        """Call after one epoch of model training to update early stopper object."""

        if val_loss < self.val_loss_min:
          self.save_checkpoint(model,val_loss)
          self.counter = 0
        else:
          self.counter = self.counter + 1
        return


            
    def save_checkpoint(self, model, val_loss):
        """Save model checkpoint.

        Args:
            model (nn.Module): Model of which parameters should be saved.
        """
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.4f} --> {val_loss:.4f}).  Saving model ...')

        self.val_loss_min = val_loss
        torch.save({
                  'model_state_dict': model.state_dict(),
                   }, self.path)
        return

 
        
    def load_checkpoint(self, model):
        """Load model from checkpoint.

        Args:
            model (nn.Module): Model that should be reset to parameters loaded
                from checkpoint.

        Returns:
            nn.Module: Model with parameters from checkpoint
        """
        if self.verbose:
            print(f'Loading model from last checkpoint with validation loss {self.val_loss_min:.4f}')
        checkpoint = torch.load(self.path)
        model.load_state_dict(checkpoint['model_state_dict'])

        
        return model

In [ ]:
'Use RMSE loss function'
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-8):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps
        
    def forward(self,yhat,y):
        loss = torch.sqrt(self.mse(yhat,y) + self.eps)
        return loss

In [ ]:
# Define Dataloader
batch_size = 32
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True,drop_last=False)
val_dataloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False,drop_last=False)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False,drop_last=False)


In [ ]:
# instantiate model and optimizer
hidden_dim = 100
num_layers = 3
lr = 0.0005
drop_out = 0.2
model = LSTM(input_dim = 15, hidden_dim = hidden_dim, drop_out = drop_out, num_layers=num_layers, pred_length = pred_length).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# run training
num_epochs = 100
patience = 10
stopper = EarlyStopper(patience = patience)
loss_function = RMSELoss()
model, train_losses, val_losses,fitted_values = run_training(model, optimizer, loss_function, num_epochs, 
                train_dataloader,val_dataloader, verbose=True, early_stopper = stopper)

stop_point_loss = stopper.val_loss_min
stop_values = (val_losses.index(stop_point_loss)+1, stop_point_loss)

display(stop_values[0])



# plot results
plot("Loss vs. Epoch", "Loss", train_losses, val_losses, extra_pt = stop_values,
   extra_pt_label = 'stopping point',yscale='linear')

#plot('Fitted values for validation set',  )
